# Import libraries 

In [1]:
import requests
from bs4 import BeautifulSoup
from itertools import cycle
import csv 

# Get Proxies 

In [2]:
def getProxies(inURL):
    
    page = requests.get(inURL)
    soup = BeautifulSoup(page.text, 'html.parser')
    terms = soup.find_all('tr')
    IPs = []

    for x in range(len(terms)):  
        
        term = str(terms[x])        
        
        if '<tr><td>' in str(terms[x]):
            pos1 = term.find('d>') + 2
            pos2 = term.find('</td>')

            pos3 = term.find('</td><td>') + 9
            pos4 = term.find('</td><td>US<')
            
            IP = term[pos1:pos2]
            port = term[pos3:pos4]
            
            if '.' in IP and len(port) < 6:
                IPs.append(IP + ":" + port)
                #print(IP + ":" + port)

    return IPs 


#Cycle through the proxies and get one to use 
proxyURL = "https://www.us-proxy.org/"
pxs = getProxies(proxyURL)
proxyPool = cycle(pxs)

In [18]:
pxs[0:10]

['50.4.60.98:80',
 '3.87.226.254:80',
 '198.199.86.11:8080',
 '138.68.60.8:8080',
 '209.97.150.167:8080',
 '191.96.42.80:8080',
 '102.129.249.120:8080',
 '3.92.176.124:80',
 '162.144.106.161:3838',
 '212.87.220.2:3128']

# Load  Webpage

In [4]:
url = 'https://www.indeed.com/jobs?q=software+engineer+intern&l=New+York%2C+NY'

#Get all of the code of the website 
page = requests.get(url, proxies = {"http": next(proxyPool)})
soup = BeautifulSoup(page.text, 'html.parser')

# Get all job postings 

In [6]:
# All the job postings are inside the td tag with the id resultsCol 
jobs = soup.find(id='resultsCol')

#Each individual job posting is represented by a div with the class jobsearch-SerpJobCard
job_elems = jobs.find_all('div', class_='jobsearch-SerpJobCard')

In [ ]:
job_elems

# Loop through job postings and extract info 

In [8]:
#Extract individual job info from job_elems, which is a list containing all the job listings on the page 
for desc in job_elems:
    
    #Extract only the job title 
    title = desc.find('h2', class_='title').text 
    
    #Get rid of uncessary whitespace and the word 'new'
    title = title.replace('new', '').replace('\n', '') 
    
    #Company name 
    company = desc.find('span', class_='company').text
    company = company.replace('\n', '')
    
    
    #Not all companies have a rating 
    #If they don't have a rating, set rating to '-'
    try:
        rating = desc.find('span', class_='ratingsContent').text
        rating = rating.replace('\n', '')
    except:
        rating = '-'
    
    
    #Not all job postings have a listed location
    #If they don't have a rating, set rating to '-'
    try:
        location = desc.find('span', class_='location').text
    except:
        location = '-'
    
    
    #Details of the job responsibilites 
    summary = desc.find('div', class_='summary').text
    summary = summary.replace('\n', '')
    
    
    #Extract direct URL to the job listing 
    rawLink = desc.find('a', class_='jobtitle')
    jobLink = rawLink.get('href')
    jobLink = jobLink[jobLink.find('?') + 1 : ]
    jobLink = ('https://www.indeed.com/viewjob?' + jobLink).replace('\n', '')
    
    
    #Some URLs are different 
    if 'jk=' not in jobLink:
        rawLink = str(rawLink.get('href'))
        jobLink = 'https://www.indeed.com/viewjob?cmp=' + company + '&t=' + rawLink[rawLink.find('jobs') + 5 : rawLink.rindex('-')] + '&jk=' + rawLink[rawLink.rindex('-') + 1 : rawLink.find('?')]

        
    print(title, company, rating)
    print(location)
    print(summary)
    print(jobLink, '\n' * 2)
    
print(len(job_elems))

Software Engineer Intern Balyasny 4.0
New York, NY
Participate in all parts of software development lifecycle – you will take part of the design, implementation, and rollout of production systems.
https://www.indeed.com/viewjob?jk=8ff009504101ef2f&fccid=3187d6d05b329ebf&vjs=3 


Software Engineer Intern PayPal 3.9
New York, NY 10018 (Clinton area)
Code high-volume and scalable software.We have opportunities in a wide range of areas including development, design, search, platform, test, quality, big data,…
https://www.indeed.com/viewjob?jk=e01674d33559a721&fccid=978d9fd9799d55a8&vjs=3 


Software Engineer Intern (Paid) at a VC-Funded Startup (Fall... Formula -
New York, NY 10010 (Flatiron District area)
You will work side by side with our product and engineering teams on a diverse set of challenges.A strong fundamental understanding of Computer Science through…
https://www.indeed.com/viewjob?cmp=Formula&t=Software-Engineer-Intern&jk=145fb24b09799861 


2020 Software Engineer Intern Bloo

# Putting it all together

In [9]:
def extractJobListings(url):
    
    page = requests.get(url, proxies = {"http": next(proxyPool)})
    soup = BeautifulSoup(page.text, 'html.parser')

    jobs = soup.find(id='resultsCol')
    job_elems = jobs.find_all('div', class_='jobsearch-SerpJobCard')
    
    #For Excel File 
    rows = []

    for desc in job_elems:

        #Extract only the job title 
        title = desc.find('h2', class_='title').text 

        #Get rid of uncessary whitespace and the word 'new'
        title = title.replace('new', '').replace('\n', '') 

        #Company name 
        company = desc.find('span', class_='company').text
        company = company.replace('\n', '')


        #Not all companies have a rating 
        try:
            rating = desc.find('span', class_='ratingsContent').text
            rating = rating.replace('\n', '')
        except:
            rating = '-'


        #Not all job postings have a listed location
        try:
            location = desc.find('span', class_='location').text
        except:
            location = '-'


        #Details of the job responsibilites 
        summary = desc.find('div', class_='summary').text
        summary = summary.replace('\n', '')


        #Extract direct URL to the job listing 
        rawLink = desc.find('a', class_='jobtitle')
        jobLink = rawLink.get('href')
        jobLink = jobLink[jobLink.find('?') + 1 : ]
        jobLink = ('https://www.indeed.com/viewjob?' + jobLink).replace('\n', '')


        #Some URLs are different 
        if 'jk=' not in jobLink:
            rawLink = str(rawLink.get('href'))
            jobLink = 'https://www.indeed.com/viewjob?cmp=' + company + '&t=' + rawLink[rawLink.find('jobs') + 5 : rawLink.rindex('-')] + '&jk=' + rawLink[rawLink.rindex('-') + 1 : rawLink.find('?')]
            
            
        rows.append([title, company, rating, location, summary, jobLink])
    
    return rows

In [10]:
#Get the total amount of pages to scrape through for all job listings found 
def getNumListings(url):
    
    page = requests.get(url, proxies = {"http": next(proxyPool)})
    soup = BeautifulSoup(page.text, 'html.parser')

    rawNumJobs = soup.find(id='searchCountPages').text.replace('\n', '')
    numJobs = rawNumJobs[rawNumJobs.find('of') + 3 : rawNumJobs.find('jobs')]
    
    pages = (int(numJobs) // 15) + 1
    return numJobs, pages    

In [15]:
#Data for each row of the excel file will be stored in this 
csvRows = []

def clean(stng):
    
    #We want to get rid of these characters 
    bad_chars = ['[', ']', '"', ','] 

    for i in bad_chars : 
        stng = stng.replace(i, '') 
        
    return stng

def scrapeListings(position):
    
    position_ = position.replace(' ', '+')
    page = 0
    
    totalJobs, pages = getNumListings('https://www.indeed.com/jobs?q=' + position_ + '&l=New+York%2C+NY')

    print(totalJobs + 'jobs found!', str(pages) + ' pages')
    
    #Go through 10 pages of results 
    for x in range(pages):

        url = 'https://www.indeed.com/jobs?q=' + position_ + '&l=New+York%2C+NY&start=' + str(page) 
        jobInfo = extractJobListings(url)

        csvRows.append(jobInfo)

        page += 10
        
        
    # Column Names in the excel file  
    fields = 'Title, Company, Rating, Location, Summary, Job Link\n' 

    # Name of Excel file  
    fileName = position + " Job Postings.csv"
    
    #Write to excel file 
    MyFile = open(fileName, 'w', encoding="utf-8")

    MyFile.write(fields)
    
    #Append the data to the rows of the file 
    for row in csvRows:
        for job in row:
            MyFile.write(clean(job[0]) + ',' + clean(job[1]) + ',' + clean(job[2]) + ',' + clean(job[3]) + ',' + clean(job[4]) + ',' + clean(job[5]))
            MyFile.write('\n')

    MyFile.close()

    print('\nSaved as ' + fileName)



In [19]:
#Job Position you are searching for 
position = 'Software Engineer Intern'

scrapeListings(position)

83 jobs found! 6 pages

Saved as Software Engineer Intern Job Postings.csv
